# TeamDynamix Open Tickets Over Time

## Get graphing libraries loaded

I'm a big fan of plot.ly, but there are many to be considered

In [139]:
import plotly
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
import cufflinks as cf
cf.go_offline()

## Use the SuperFine Cedarville University TeamDynamix python library to collect the tickets

In [140]:
import tdxlib
tdx = tdxlib.tdx_ticket_integration.TDXTicketIntegration()

In [141]:
from dateutil.parser import parse

start = '2018-02-01'
end = '2019-06-19'

criteria = {'CreatedDateFrom': start,
        'CreatedDateTo': end}
tklist = tdx.search_tickets(criteria, max_results=0, closed=False, cancelled=False, other_status=False)

## Use pandas to collect all of the dates a ticket is open

In [142]:
from datetime import datetime
import pytz 
from pytz import reference
import pandas as pd
import numpy as np

datecollection = None

for i, tk in enumerate(tklist):
    localtime = reference.LocalTimezone()
    cr = tk.get_attribute('CreatedDate').replace(tzinfo=pytz.utc).date()
    cp = tk.get_attribute('CompletedDate').replace(tzinfo=pytz.utc).date()
    
    if cp < cr:
        cp = datetime.now(pytz.utc).date()      

    range = pd.date_range(start=cr, end=cp)
    if i is 0:
        datecollection = range
    else:
        datecollection = datecollection.append(range)    

## Sort and then chart the results

In [143]:
datecounts = datecollection.value_counts(sort=False).sort_index()

figure = datecounts.iplot(asFigure=True,
                kind='scatter',xTitle='Dates',yTitle='Open Tickets',title='Open Tickets over Time')
iplot(figure)

